time series 
product stats/peak sale time
product waste
store stats
customer order predictions 
sales targets/met?
employee audit/stats                                                                                                                                                                                                                                                                                                                                                                      

time series 
product stats/peak sale time
product waste
store stats
customer order predictions 
sales targets/met?
employee audit/stats                                                                                                                                                                                                                                                                                                                                                                      

In [1]:
import pandas as pd
import numpy as np

import matplotlib
import seaborn as sns

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import pearsonr, spearmanr
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('201904 sales reciepts.csv')

In [3]:
df.loc[123]

transaction_id             575
transaction_date    2019-04-01
transaction_time      17:41:55
sales_outlet_id              3
staff_id                    17
customer_id                269
instore_yn                   N
order                        1
line_item_id                 1
product_id                  55
quantity                     2
line_item_amount           8.0
unit_price                 4.0
promo_item_yn                N
Name: 123, dtype: object

In [4]:
df[df.order != 1]

,transaction_id,transaction_date,transaction_time,sales_outlet_id,staff_id,customer_id,instore_yn,order,line_item_id,product_id,quantity,line_item_amount,unit_price,promo_item_yn
10193,5,2019-04-07,09:59:06,5,29,0,Y,5,1,73,1,3.75,3.75,N
10194,5,2019-04-07,09:59:06,5,29,0,Y,9,9,9,1,28.00,28.00,N
10198,7,2019-04-07,08:35:35,5,14,0,Y,5,1,77,1,3.00,3.00,N
10202,10,2019-04-07,07:37:21,5,14,5256,Y,5,1,71,1,3.75,3.75,N
10205,11,2019-04-07,09:27:50,5,14,5598,N,9,9,9,1,23.00,23.00,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46217,424,2019-04-27,11:24:27,5,29,5227,N,7,1,3,1,14.75,14.75,N
46228,434,2019-04-27,15:11:37,5,27,5219,Y,9,1,82,1,12.00,12.00,N
46261,459,2019-04-27,13:16:07,5,7,5210,Y,7,1,3,1,14.75,14.75,N
46266,462,2019-04-27,15:28:27,5,27,5830,N,7,1,19,1,6.40,6.40,N


 - FIX TRANSACTION IDs
 - DROP line item id (numbers not making sense)

In [5]:
df[df.transaction_id == 575]

,transaction_id,transaction_date,transaction_time,sales_outlet_id,staff_id,customer_id,instore_yn,order,line_item_id,product_id,quantity,line_item_amount,unit_price,promo_item_yn
123,575,2019-04-01,17:41:55,3,17,269,N,1,1,55,2,8.00,4.00,N
124,575,2019-04-01,17:41:55,3,17,269,N,1,5,71,1,3.75,3.75,N
5674,575,2019-04-04,13:44:15,5,12,5236,Y,1,1,31,2,4.40,2.20,N
9252,575,2019-04-06,09:27:30,8,15,8005,N,1,1,39,1,4.25,4.25,N
9253,575,2019-04-06,09:27:30,8,15,8005,N,1,5,74,1,3.50,3.50,N
26377,575,2019-04-16,08:49:22,5,12,0,Y,1,1,23,3,7.50,2.50,N
27631,575,2019-04-17,09:44:32,3,20,0,Y,1,1,40,1,3.75,3.75,N
27632,575,2019-04-17,09:44:32,3,20,0,Y,1,2,84,1,0.80,0.80,N
28862,575,2019-04-17,11:01:41,8,44,0,Y,1,1,27,2,7.00,3.50,N
28863,575,2019-04-17,11:01:41,8,44,0,Y,1,5,73,1,3.75,3.75,N


### Need to redo the transaction id 

In [6]:
df.transaction_id = 0
df.transaction_id = df.transaction_id.astype(int)

In [7]:
id = 1
for current_row in range(len(df)):
    if df.loc[current_row, 'transaction_id'] == 0:
        df.loc[current_row, 'transaction_id'] = id
        for row in range(current_row, len(df)):
            if df.loc[current_row, 'transaction_date'] == df.loc[row, 'transaction_date']:
                if (df.loc[current_row, 'transaction_time'] == df.loc[row, 'transaction_time'] ) & (df.loc[current_row, 'sales_outlet_id'] == df.loc[row, 'sales_outlet_id']):
                    df.loc[row, 'transaction_id'] = id
            else:
                break
        id+=1
    else:
        pass
        

In [8]:
(df.loc[28, 'transaction_time'] == df.loc[29, 'transaction_time']) & (df.loc[28, 'transaction_date'] == df.loc[29, 'transaction_date']) & (df.loc[28, 'sales_outlet_id'] == df.loc[29, 'sales_outlet_id'])


True

### Columns to drop
- line_item_id
- order

In [9]:
df.drop(columns=['line_item_id', 'order'], inplace=True)

In [10]:
df

,transaction_id,transaction_date,transaction_time,sales_outlet_id,staff_id,customer_id,instore_yn,product_id,quantity,line_item_amount,unit_price,promo_item_yn
0,1,2019-04-01,12:04:43,3,12,558,N,52,1,2.50,2.50,N
1,2,2019-04-01,15:54:39,3,17,781,N,27,2,7.00,3.50,N
2,3,2019-04-01,14:34:59,3,17,788,Y,46,2,5.00,2.50,N
3,4,2019-04-01,16:06:04,3,12,683,N,23,2,5.00,2.50,N
4,5,2019-04-01,19:18:37,3,17,99,Y,34,1,2.45,2.45,N
...,...,...,...,...,...,...,...,...,...,...,...,...
49889,39021,2019-04-29,16:51:58,8,42,0,N,30,1,3.00,3.00,N
49890,39022,2019-04-29,16:51:14,8,42,8412,Y,25,2,4.40,2.20,N
49891,39023,2019-04-29,11:17:36,8,15,0,Y,31,1,2.20,2.20,N
49892,39024,2019-04-29,15:45:52,8,45,8030,N,44,2,5.00,2.50,N


## Change in store column to 1 or 0 insteads of Y or N

In [11]:
df['instore_yn'] = df['instore_yn'].replace({'Y': 1, 'N': 0})


In [12]:
df = df.rename(columns={'instore_yn': 'is_instore'})

In [13]:
df.is_instore.value_counts()

1    24992
0    24608
       294
Name: is_instore, dtype: int64

## Change promo item column to 1 or 0 insteads of Y or N

In [14]:
df['promo_item_yn'] = df['promo_item_yn'].replace({'Y': 1, 'N': 0})

In [15]:
df = df.rename(columns={'promo_item_yn': 'is_promo'})

In [16]:
df

,transaction_id,transaction_date,transaction_time,sales_outlet_id,staff_id,customer_id,is_instore,product_id,quantity,line_item_amount,unit_price,is_promo
0,1,2019-04-01,12:04:43,3,12,558,0,52,1,2.50,2.50,0
1,2,2019-04-01,15:54:39,3,17,781,0,27,2,7.00,3.50,0
2,3,2019-04-01,14:34:59,3,17,788,1,46,2,5.00,2.50,0
3,4,2019-04-01,16:06:04,3,12,683,0,23,2,5.00,2.50,0
4,5,2019-04-01,19:18:37,3,17,99,1,34,1,2.45,2.45,0
...,...,...,...,...,...,...,...,...,...,...,...,...
49889,39021,2019-04-29,16:51:58,8,42,0,0,30,1,3.00,3.00,0
49890,39022,2019-04-29,16:51:14,8,42,8412,1,25,2,4.40,2.20,0
49891,39023,2019-04-29,11:17:36,8,15,0,1,31,1,2.20,2.20,0
49892,39024,2019-04-29,15:45:52,8,45,8030,0,44,2,5.00,2.50,0


In [18]:
df.loc[11067]

transaction_id            9337
transaction_date    2019-04-07
transaction_time      17:33:57
sales_outlet_id              8
staff_id                    43
customer_id                  0
is_instore                    
product_id                  53
quantity                     2
line_item_amount           6.0
unit_price                 3.0
is_promo                     0
Name: 11067, dtype: object

In [ ]:
df.is_promo = df.is_promo.astype(int)

In [ ]:
df

In [ ]:
df.customer_id = df.customer_id.astype(int)

In [ ]:
df.sales_outlet_id = df.sales_outlet_id.astype(int)

In [ ]:
df.product_id = df.product_id.astype(int)

In [ ]:
df.quantity = df.quantity.astype(int)

In [ ]:
df.to_csv('clean_sales.csv', index=False)a